# 4. Geocoding & table joins

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')

In [3]:
from geopy.geocoders import Nominatim

In [4]:
geolocator = Nominatim(user_agent='kaggle_learn')
location = geolocator.geocode("Pyramid of Khufu")

print(location.point)
print(location.address)

29 58m 44.9927s N, 31 8m 3.16774s E
هرم خوفو, شارع ابو الهول السياحي, كوم الأخضر, الجيزة, 12125, مصر


In [5]:
point = location.point
print("Latitude :", point.latitude)
print("Longitude :", point.longitude)

Latitude : 29.97916465
Longitude : 31.13421326082681


## Top 100 universities in Europe

In [6]:
universities = pd.read_csv('C:/Users/GBOOK/PythonGeoSpatialAnalysis/archive/top_universities.csv')
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [7]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude' : point.latitude, 'Longitude' : point.longitude})
    except:
        return None
    
universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of address were geocoded!".format((1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.csrs = {'init' : 'epsg:4326'}
universities.head()

90.0% of address were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.210946,0.092005,POINT (0.092 52.21095)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179)


## visualize universities in the Europ

In [8]:
# Create a map
m = folium.Map(loacation=[54, 15], tiles = 'openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)
    
m

## Table join

In [14]:
world = gpd.read_file('C:/Users/GBOOK/PythonGeoSpatialAnalysis/110m_cultural/ne_110m_admin_0_countries.shp')
europe = world.loc[world.CONTINENT == 'Europe'].reset_index(drop=True)

europe_stats = europe[["NAME", "POP_EST", "GDP_MD"]]
europe_boundaries = europe[["NAME", "geometry"]]

In [15]:
europe_boundaries.head()

,NAME,geometry
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571..."
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80..."
2,France,"MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3...."
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239..."
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834..."


In [16]:
europe_stats.head()

,NAME,POP_EST,GDP_MD
0,Russia,144373535.0,1699876
1,Norway,5347896.0,403336
2,France,67059887.0,2715518
3,Sweden,10285453.0,530883
4,Belarus,9466856.0,63080


## Attribute join

In [17]:
europe = europe_boundaries.merge(europe_stats, on="NAME")
europe.head()

,NAME,geometry,POP_EST,GDP_MD
0,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571...",144373535.0,1699876
1,Norway,"MULTIPOLYGON (((15.14282 79.67431, 15.52255 80...",5347896.0,403336
2,France,"MULTIPOLYGON (((-51.6578 4.15623, -52.24934 3....",67059887.0,2715518
3,Sweden,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",10285453.0,530883
4,Belarus,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",9466856.0,63080


## Spatial Join

In [20]:
european_universities = gpd.sjoin(universities, europe)

print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.NAME.unique())))

european_universities.head()

We located 90 universities.
Only 87 of the universities were located in Europe (in 15 different countries).


,Name,Latitude,Longitude,geometry,index_right,NAME,POP_EST,GDP_MD
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871),28,United Kingdom,66834405.0,2829108
1,University of Cambridge,52.210946,0.092005,POINT (0.092 52.21095),28,United Kingdom,66834405.0,2829108
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896),28,United Kingdom,66834405.0,2829108
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322),19,Switzerland,8574832.0,703082
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179),28,United Kingdom,66834405.0,2829108
